# Watershed

Import dependencies

In [1]:
import glob
import cv2
import numpy as np

Load images

In [2]:
images = glob.glob("./*/*/*/[0-9]*[0-9].png")

Apply watershed to each image an save the mask

In [5]:
save_name = "watershed.png"

for image in images:
    #Construct save path
    save_path = image.replace(image.split("\\")[-1], "")

    #Load image to memory in color and grayscale
    cached_image = cv2.imread(image)
    cached_image_gray = cv2.imread(image, cv2.IMREAD_GRAYSCALE)

    #Detect sensor and set threshold value accordingly
    threshold_value = 54.75 if "FX10" in image else 93.47

    #Apply thresholding to the image
    threshold_result = cached_image_gray > threshold_value

    #Noise removal
    kernel_11 = np.ones((11,11), np.uint8)
    opening = cv2.morphologyEx(threshold_result.astype(np.uint8) * 255, cv2.MORPH_OPEN, kernel_11, iterations = 4)

    #Finding sure background area
    sure_bg = cv2.dilate(opening, kernel_11, iterations = 5)

    #Finding sure foreground area
    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
    ret, sure_fg = cv2.threshold(dist_transform, 0.03 * dist_transform.max(), 255, 0)

    #Finding unknown region
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg, sure_fg)

    #Marker labelling
    ret, markers = cv2.connectedComponents(sure_fg, connectivity = 8)

    #Add one to all labels so that sure background is not 0, but 1
    markers = markers + 1
    #Mark the region of unknown with zero and apply watershed
    markers[unknown == 255] = 0
    markers = cv2.watershed(cached_image, markers)

    #Mark every pixel with 1 value as background
    result = markers != 1

    #Save the binary image
    cv2.imwrite('{}{}'.format(save_path, save_name), result.astype(np.uint8)*255)